In [1]:
import os
import time

from dotenv import load_dotenv
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [25]:
#
# ----------------

# step0. uc.Chrome()로 크롤링 할 driver객체 만들기
wd = webdriver.Chrome()
first_tags = ['서비스업', '제조/화학', '의료/제약/복지', '유통/무역/운송', '교육업',
            '건설업', 'IT/웹/통신', '미디어/디자인', '은행/금융업', '기관/협회']
urls = [str(x) for x in range(100,1001, 100)] # IT의 2차분류

arr = []
for url in urls:
    key = first_tags[(int(url)-100)//100 - 1]
    wd.get(f'https://www.jobplanet.co.kr/companies?&industry_id={url}')
    time.sleep(3)

    dict, dict2 = {}, {}
    while 1:
        # (1)element's' & (2)section:nth-child(1)~10 -> nth-child(k)를 떼서, section만 남김
        company_infos = wd.find_elements(By.CSS_SELECTOR, '#listCompanies > div > div.section_group > section > div > div > dl.content_col2_4')
        company_names = wd.find_elements(By.CSS_SELECTOR, '#listCompanies > div > div.section_group > section > div > div > dl.content_col2_3.cominfo > dt > a')
        
        now = wd.current_url
        for info, name in zip(company_infos, company_names):
            name = name.text
            
            a = info.text.replace(',','').replace('개의 리뷰','').replace('평균','').replace(' 만원','')
            b = a.split('\n') # ['2507', '평점', '2.8', '4172']
            review, avg_rate, avg_salary = int(b[0]), float(b[2]), int(b[3])

            if review == 0:
                continue
            dict[name] = [review, avg_rate, avg_salary]
        
        # elemen't' - elements하면 x
        wd.find_element(By.CSS_SELECTOR, '#listCompanies > div > div.pg_bottom.um_paginnation > article > a.btn_pgnext').click()
        time.sleep(2)
        
        if now == wd.current_url:
            break
    
    dict2[key] = dict
    arr.append(dict2)
    print(len(dict2[key].keys())) # 누락 데이터 체크용
    
print(arr, len(arr))

1732
5989
968
3151
643
1149
120
1974
600
1247
[{'기관/협회': {'(주)에스씨케이컴퍼니': [4593, 3.3, 3194], '파리바게뜨 (매장)': [4197, 3.4, 3016], '롯데리아 (매장)': [3081, 3.3, 3098], '한국맥도날드(유)': [2936, 3.4, 3218], '(주)트랜스코스모스코리아': [2253, 2.6, 2762], '(주)유베이스': [2099, 2.6, 2719], '(주)하나투어': [1800, 3.0, 3397], '(주)세스코': [1597, 3.0, 3502], '(주)에스텍시스템': [1595, 2.4, 2779], '(주)유니에스': [1529, 2.6, 2758], '(주)아워홈': [1480, 2.7, 4103], '(주)에스케이쉴더스': [1480, 2.6, 3745], '씨제이푸드빌(주)': [1468, 2.9, 3847], '(주)커피빈코리아': [1456, 2.8, 2601], '애슐리 (매장)': [1424, 3.0, 2924], '(주)비케이알': [1416, 3.0, 3068], '(주)호텔신라': [1281, 3.0, 4124], '빕스 (매장)': [1206, 3.4, 2392], '(주)에스원': [1111, 2.8, 3838], '엔제리너스 (매장)': [1074, 3.3, 2575], '(주)삼구아이앤씨': [1046, 2.7, 3613], '롯데월드': [995, 3.4, 4065], '(주)메타넷엠플랫폼': [963, 2.4, 2670], '아웃백스테이크하우스코리아(유)': [946, 3.3, 2874], '이트너스(주)': [931, 2.6, 3435], '코웨이(주)': [926, 3.0, 5333], '뚜레쥬르 (매장)': [897, 3.4, 2776], '던킨/던킨도너츠 (매장)': [881, 3.3, 2782], '(주)제니엘': [769, 2.8, 2835], '더케이텍(주)': [769, 2.4, 3114], '한국마사회'

In [ ]:
# 1732
# 5989
# 968
# 3151
# 643
# 1149
# 120
# 1974
# 600
# 1247

In [27]:
import pandas as pd

english_first_tags = ['SERVICE', 'MANUFAC_CHEMI', 'MEDICAL', 'DISTRI_TRADE',
            'EDUCATION', 'CONSTRUCTION', 'IT', 'MEDIA', 'FINANCIAL', 'ASSOCIATION']
column_list = ['리뷰수(개)', '평점', '평균 연봉(만)']

# for k in range(len(first_tags)):
for k in range(0,10):
    tag = arr[k][first_tags[k]]

    data_for_df = []
    for key, value in tag.items():
        data_for_df.append(value)

    df = pd.DataFrame(data_for_df, index=tag.keys(), columns=column_list)
    
    print(df.tail())

    df.to_csv(f'1-{k}.{english_first_tags[k]}.csv', encoding='utf-8-sig')


# todo 근데, 여기서 또 평균 연봉이 0인게 있음 -> 입력 안한듯 -> NaN인셈 치고, 보안/백신계 평균연봉 넣어주기 걍(결측치 처리)



KeyError: '서비스업'

In [17]:
arr[0].keys()

dict_keys(['기관/협회'])